## Exporting agriculture patents

In [ ]:
from flashtext import KeywordProcessor
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Time of execution
start = time.time()

# catalogue = pd.read_csv('catalogue.csv')
# catalogue.fillna(method='ffill', inplace = True)
monitoring_terms = ['agriculture']# list(catalogue.keywords)

print('terms to use for monitoring: {}'.format(len(monitoring_terms)))

patents = pd.read_csv('http://data.patentsview.org/20190312/download/patent.tsv.zip',
                      sep='\t', error_bad_lines=False, verbose=False)


print('total patents database: {}'.format(len(patents)))

def extract(vec, dictionary, info=False):
    matrix = []
    for line in vec:
        matrix.append(dictionary.extract_keywords(str(line), span_info=info))
    return matrix

#proccess
data = patents
dictionary = KeywordProcessor()
dictionary.add_keywords_from_list(monitoring_terms)
extracted = extract(data.abstract, dictionary)
row = [list(set(i)) if len(i)>0 else '' for i in extracted]
data['matches'] = [str(i).replace('[', '').replace(']', '') for i in row]
data['count_matches'] = [len(i) for i in extracted]
data['count_unique_matches'] = [len(set(i)) for i in extracted]

#get the data matches
data_matches = data[data['count_matches'] > 0]
data_matches['date_str'] = ["".join(i.split('-')) for i in data_matches.date]
links = []
for i in data_matches.index:
    line = data_matches.loc[i]
    link = "https://worldwide.espacenet.com/publicationDetails/biblio?"+\
    "CC="+str(line.country)+\
    "&NR="+str(line.id)+\
    "&KC="+str(line.kind)+\
    "&date="+str(line.date_str)+\
    "&locale=en_EP"
    links.append(link)
data_matches['link'] = links
print('number of matches found: {}, {}%'.format(len(data_matches), round(100*len(data_matches)/len(data),5)))

#export data
export = data_matches.sort_values('count_matches', ascending=False)
export = export[['title','link','abstract','matches']].drop_duplicates(subset ="link")
export.columns = ['name','url','description','topic']
export.to_excel('agripatents.xlsx')
print('Exported in agripatents.xlsx')
end = time.time()
print('Elapsed time: {}'.format(time.strftime("%H:%M:%S", time.gmtime(end - start))))

terms to use for monitoring: 1
